In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
import os
import math

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score
import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.neighbors import KNeighborsClassifier


In [2]:
def calculate_angle(center_landmark, start_landmark, end_landmark):

    center = np.array(center_landmark)
    start = np.array(start_landmark)
    end = np.array(end_landmark)
    

    vector1 = start - center
    vector2 = end - center
    

    angle_radians = np.arccos(np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))
    

    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees


def draw_selected_landmarks_on_image(rgb_image, landmarks, selected_landmarks, connections):

    annotated_image = np.copy(rgb_image)


    for index in selected_landmarks:
        landmark = landmarks.landmark[index]
        x, y, z = landmark.x, landmark.y, landmark.z

        x_pixel, y_pixel = int(x * annotated_image.shape[1]), int(y * annotated_image.shape[0])
        cv2.circle(annotated_image, (x_pixel, y_pixel), radius=5, color=(0, 255, 0), thickness=-1)
    for connection in connections:
        start_index, end_index = connection
        start_landmark = landmarks.landmark[start_index]
        end_landmark = landmarks.landmark[end_index]
        start_x, start_y = int(start_landmark.x * annotated_image.shape[1]), int(start_landmark.y * annotated_image.shape[0])
        end_x, end_y = int(end_landmark.x * annotated_image.shape[1]), int(end_landmark.y * annotated_image.shape[0])
        cv2.line(annotated_image, (start_x, start_y), (end_x, end_y), (0, 255, 0), 2)
            

    return annotated_image

In [3]:
mp_pose = mp.solutions.pose
selected_landmarks_indices = [16,14,12,24,26,28,32]
selected_landmarks_connections = [(16,14),(14,12),(24,26),(26,28),(12,24),(24,28),(28,32)]

In [4]:
data=pd.read_csv('ankle-train.csv')

In [5]:
X = data[['angle']]
y = data['label']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

clf = RandomForestClassifier(random_state=42)

In [6]:
clf.fit(X_train, y_train)

train_preds = clf.predict(X_train)
val_preds = clf.predict(X_val)
test_preds = clf.predict(X_test)

train_acc = accuracy_score(y_train, train_preds)
val_acc = accuracy_score(y_val, val_preds)
test_acc = accuracy_score(y_test, test_preds)

conf_matrix = confusion_matrix(y_test, test_preds)

precision = precision_score(y_test, test_preds)
recall = recall_score(y_test, test_preds)

print(f"Training accuracy: {train_acc:.4f}")
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print(f"\nPrecision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

Training accuracy: 1.0000
Validation accuracy: 1.0000
Test accuracy: 1.0000

Confusion Matrix:
[[124   0]
 [  0 130]]

Precision: 1.0000
Recall: 1.0000


In [7]:
joblib.dump(clf, 'sajda_ankle_RandomForest.pkl')

['sajda_ankle_RandomForest.pkl']

In [80]:
# data = pd.read_csv('knee-train.csv')

X = data[['angle']]
y = data['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model = LogisticRegression()


In [81]:
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

train_acc = accuracy_score(y_train, model.predict(X_train))
val_acc = accuracy_score(y_test, y_pred)

print('Train accuracy:', train_acc)
print('Validation accuracy:', val_acc)

conf_matrix = confusion_matrix(y_test, y_pred)
print('Confusion matrix:')
print(conf_matrix)

precision = precision_score(y_test, y_pred)
print('Precision:', precision)

recall = recall_score(y_test, y_pred)
print('Recall:', recall)

f1 = f1_score(y_test, y_pred)
print('F1-score:', f1)

Train accuracy: 0.9444741057127602
Validation accuracy: 0.9487726787620064
Confusion matrix:
[[404  48]
 [  0 485]]
Precision: 0.9099437148217636
Recall: 1.0
F1-score: 0.9528487229862475


In [82]:
joblib.dump(model, 'sajda_knee_LogisticRegression.pkl')

['sajda_knee_LogisticRegression.pkl']

In [83]:
X = data['angle'].values.reshape(-1, 1)  # Reshape to 2D array for fitting
y = data['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the SVM model
svc = SVC()
svc.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svc.predict(X_test)

# Evaluate the model's performance
train_acc = accuracy_score(y_train, model.predict(X_train))
test_acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

c:\Users\Wajdan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


In [84]:
print("Training accuracy:", train_acc)
print("Testing accuracy:", test_acc)
print("Confusion matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)

Training accuracy: 0.9466097170315003
Testing accuracy: 0.9402347918890075
Confusion matrix:
 [[397  56]
 [  0 484]]
Precision: 0.8962962962962963
Recall: 1.0


In [85]:
joblib.dump(svc, "sajda_knee_SVM.pkl")

['sajda_knee_SVM.pkl']

In [99]:
X = data[['angle']]  # Independent variable
y = data['label']    # Dependent variable

# Split the data into train, validation, and test sets (80% train, 10% validation, 10% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Initialize the K-Nearest Neighbors Classifier with k=5 (you can modify k as needed)
knn = KNeighborsClassifier(n_neighbors=7)

# Train the model
knn.fit(X_train, y_train)

# Predict on validation set
val_predictions = knn.predict(X_val)

# Calculate validation accuracy
val_acc = accuracy_score(y_val, val_predictions)
print(f'Validation Accuracy: {val_acc:.4f}')

# Predict on test set
test_predictions = knn.predict(X_test)

# Calculate test accuracy
test_acc = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_acc:.4f}')

# Calculate train accuracy
train_predictions = knn.predict(X_train)
train_acc = accuracy_score(y_train, train_predictions)
print(f'Train Accuracy: {train_acc:.4f}')

# Confusion matrix
conf_matrix = confusion_matrix(y_test, test_predictions)
print('Confusion Matrix:')
print(conf_matrix)

# Precision and recall
precision = precision_score(y_test, test_predictions)
recall = recall_score(y_test, test_predictions)
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')

Validation Accuracy: 0.9338
Test Accuracy: 0.9467
Train Accuracy: 0.9463
Confusion Matrix:
[[186  25]
 [  0 258]]
Precision: 0.9117
Recall: 1.0000


In [100]:
joblib.dump(knn, "sajda_knee_knn.pkl")

['sajda_knee_knn.pkl']